## Week 2 Day 2 <center> Sales Agent

### We are going to build a simple sales agent that will:

1. Be able to send email
2. Use different tools created by us and
3. Hand off task


We would be creating SENDGRID_API_KEY using its free account for our agents to be able to send emails

In [1]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
# # Let's just check emails are working for you

# def send_test_email():
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
#     to_email = To("ed.donner@gmail.com")  # Change to your recipient
#     content = Content("text/plain", "This is an important test email")
#     mail = Mail(from_email, to_email, "Test email", content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     print(response.status_code)

# send_test_email()

In [4]:
def send_test_email():
    # Debug: Check if environment variables are loaded
    api_key = os.environ.get('SENDGRID_API_KEY')
    from_email_addr = os.getenv("YAHOO_ID")
    to_email_addr = os.getenv("GMAIL_ID")
    
    print(f"API Key exists: {api_key is not None}")
    # print(f"From email: {from_email_addr}")
    # print(f"To email: {to_email_addr}")
    
    if not api_key:
        raise ValueError("SENDGRID_API_KEY environment variable is not set")
    if not from_email_addr:
        raise ValueError("YAHOO_ID environment variable is not set")
    if not to_email_addr:
        raise ValueError("GMAIL_ID environment variable is not set")
    
    try:
        # Initialize SendGrid client
        sg = sendgrid.SendGridAPIClient(api_key=api_key)
        
        # Create the email message - Fixed: Remove .get() method call
        message = Mail(
            from_email=from_email_addr,  # Can pass string directly
            to_emails=to_email_addr,     # Can pass string directly
            subject="Test email from your AI Agent",
            plain_text_content="This is an important test email from your AI Agent"
        )
        
        # Alternative way using the helper classes (if you prefer):
        # from_email = Email(from_email_addr)
        # to_email = To(to_email_addr)
        # content = Content("text/plain", "This is an important test email from your AI Agent")
        # message = Mail(from_email, to_email, "Test email from your AI Agent", content)
        
        # Send the email
        response = sg.send(message)  # Simplified method call
        
        print(f"Email sent successfully!")
        print(f"Status code: {response.status_code}")
        print(f"Response body: {response.body}")
        print(f"Response headers: {response.headers}")
        
    except Exception as e:
        print(f"Error sending email: {str(e)}")
        if hasattr(e, 'body'):
            print(f"Error body: {e.body}")


send_test_email()

API Key exists: True
Email sent successfully!
Status code: 202
Response body: b''
Response headers: Server: nginx
Date: Fri, 22 Aug 2025 20:35:32 GMT
Content-Length: 0
Connection: close
X-Message-Id: VdG4oG2lSXW71bDwWaHLjw
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




# Agent Workflow

#### Creating multiple sales agent that writes emails

In [5]:
sag1 = "You are a sales agent working at ComplAI,\
a company that provides SaaS tool for SOC 2 compliance and preparing for audits, powered by AI.\
You write professional serious cold emails."

    
sag2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

sag3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."
    

In [6]:
sales_agent1 = Agent(
    name="Sales Agent 1",
    instructions=sag1,
    model="gpt-4o-mini")


sales_agent2 = Agent(
    name="Sales Agent 2",
    instructions=sag2,
    model="gpt-4o-mini")

sales_agent3 = Agent(
    name="Sales Agent 3",
    instructions=sag3,
    model="gpt-4o-mini")

In [7]:
sales_agent1


Agent(name='Sales Agent 1', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working at ComplAI,a company that provides SaaS tool for SOC 2 compliance and preparing for audits, powered by AI.You write professional serious cold emails.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [8]:
result = await Runner.run(sales_agent1, input="Write a cold sales email")


In [9]:
# Understand the below run.streamed implementation instead of Runner.run

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I'm reaching out from ComplAI, a company dedicated to streamlining SOC 2 compliance and audit processes through the power of artificial intelligence.

Navigating SOC 2 requirements can be complex and time-consuming. Our SaaS tool automates key compliance tasks, ensuring you stay organized and prepared for audits while significantly reducing manual effort. With ComplAI, you can:

- **Effortlessly manage documentation and evidence**: Our platform simplifies the storage and retrieval of necessary documents, making the audit process seamless.
- **Stay up-to-date with compliance changes**: We'll notify you of any regulations or updates relevant to your industry.
- **Enhance collaboration across your team**: Our intuitive interface promotes visibility and accessibility among team members.

I’d love the opportunity to demonstrate how ComplAI can add val

##### Now creating traces for tracking conversation flow for historical purposes

In [10]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m with ComplAI, a company dedicated to simplifying SOC 2 compliance with our advanced AI-driven SaaS tool.

Navigating the complexities of SOC 2 compliance can be a daunting task. Our platform is designed to save you time and reduce stress by automating key processes, helping you prepare for audits with greater efficiency and accuracy.

Here’s what we offer:
- **Automated Documentation:** Quickly generate and manage the documentation needed for SOC 2 compliance.
- **Audit Preparation:** Streamline your audit processes, ensuring you’re always prepared for assessments.
- **Real-Time Insights:** Leverage AI to gain actionable insights and stay ahead of compliance requirements.

I would love to discuss how ComplAI could enhance your compliance strategy. Are you available for a brief call this week?

Looking forward to your thoughts.

Best regards

Traces for the run above can be found on: 

https://platform.openai.com/traces

## Agent and Tool integration

Instead of writing lengthy Json, we can now convert any functions into tool by using decorator @function_tool; look at the example below:

In [11]:
@function_tool
def send_test_email():
    # Debug: Check if environment variables are loaded
    api_key = os.environ.get('SENDGRID_API_KEY')
    from_email_addr = os.getenv("YAHOO_ID")
    to_email_addr = os.getenv("GMAIL_ID")
    
    print(f"API Key exists: {api_key is not None}")
    # print(f"From email: {from_email_addr}")
    # print(f"To email: {to_email_addr}")
    
    if not api_key:
        raise ValueError("SENDGRID_API_KEY environment variable is not set")
    if not from_email_addr:
        raise ValueError("YAHOO_ID environment variable is not set")
    if not to_email_addr:
        raise ValueError("GMAIL_ID environment variable is not set")
    
    try:
        # Initialize SendGrid client
        sg = sendgrid.SendGridAPIClient(api_key=api_key)
        
        # Create the email message - Fixed: Remove .get() method call
        message = Mail(
            from_email=from_email_addr,  # Can pass string directly
            to_emails=to_email_addr,     # Can pass string directly
            subject="Test email from your AI Agent",
            plain_text_content="This is an important test email from your AI Agent"
        )
        
        # Alternative way using the helper classes (if you prefer):
        # from_email = Email(from_email_addr)
        # to_email = To(to_email_addr)
        # content = Content("text/plain", "This is an important test email from your AI Agent")
        # message = Mail(from_email, to_email, "Test email from your AI Agent", content)
        
        # Send the email
        response = sg.send(message)  # Simplified method call
        
        print(f"Email sent successfully!")
        print(f"Status code: {response.status_code}")
        print(f"Response body: {response.body}")
        print(f"Response headers: {response.headers}")
        
    except Exception as e:
        print(f"Error sending email: {str(e)}")
        if hasattr(e, 'body'):
            print(f"Error body: {e.body}")


In [12]:
send_test_email

FunctionTool(name='send_test_email', description='', params_json_schema={'properties': {}, 'title': 'send_test_email_args', 'type': 'object', 'additionalProperties': False, 'required': []}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x113f03920>, strict_json_schema=True, is_enabled=True)

Even an agent can be converted into tool

In [15]:
tool1 = sales_agent1.as_tool(
    tool_name="Sales Agent 1",
    tool_description="A sales agent that writes professional serious cold emails for ComplAI")
tool1

FunctionTool(name='Sales Agent 1', description='A sales agent that writes professional serious cold emails for ComplAI', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Sales Agent 1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11465d9e0>, strict_json_schema=True, is_enabled=True)

In [30]:
# Converting our functions and all the agents into tool

description = "Write a cold sales email"

sales_agents = [sales_agent1, sales_agent2, sales_agent3]

tools = []
for idx, agent in enumerate(sales_agents, start=1):
    # make a safe tool name
    safe_name = agent.name.lower().replace(" ", "_") + f"_{idx}"
    
    tool = agent.as_tool(
        tool_name=safe_name,
        tool_description=description
    )
    tools.append(tool)


tools.append(send_test_email)
print(tools) 




[FunctionTool(name='sales_agent_1_1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_1_1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115222840>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent_2_2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_2_2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115223060>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent_3_3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}

'Sales Agent 3'

In [31]:
sales_manager_instructions = """You're sales manager of the company ComplAI, you use tools given to you to help you write cold sales emails to potential clients. \
    You never write email yourself, you always use the tools given to you. \
    You always use the most appropriate tool for the job. \
    Before sending email, you use all the 3 sales agents tool to write 3 different versions of cold sales email, \
    then you pick the best one and send it using the send_test_email tool."""




sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    model="gpt-4o-mini"
)


message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)




API Key exists: True
Email sent successfully!
Status code: 202
Response body: b''
Response headers: Server: nginx
Date: Fri, 22 Aug 2025 21:01:55 GMT
Content-Length: 0
Connection: close
X-Message-Id: Bh_Geu6jRcOkP1VMfN052g
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




#### What is Handoff in Agentic workflow?

A handoff is when one agent finishes its part of the work and explicitly passes control (along with context/results) to another agent.
Think of it as “passing the baton” in a relay race.

* Tools are things an agent can call temporarily (like a calculator, or another agent wrapped as a callable). The calling agent still stays in control.

* Handoffs are when the agent says: “I’m done, someone else should now take over” — and control switches to another agent.


We will learn the above concept with the example below, by utilizing subject creator and html instruction creator agent

In [32]:
subject_instructions = "You're an expert at writing email subjects that get emails opened. \
    You write concise, engaging, and relevant subject lines that entice recipients to open the email based on the email text provided"

subject_agent = Agent(
    name="Subject Agent",
    instructions=subject_instructions,
    model="gpt-4o-mini"
)

subject_tool = subject_agent.as_tool(
    tool_name="subject_writer",
    tool_description="Writes engaging email subject lines based on the email content provided"
)

html_converter_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

html_converter_agent = Agent(
    name="HTML Converter Agent",
    instructions=html_converter_instructions,
    model="gpt-4o-mini"
)

html_converter_tool = html_converter_agent.as_tool(
    tool_name="html_converter",
    tool_description="Converts a text email body to an HTML email body"
)


handoff_tools = [subject_tool, html_converter_tool, send_test_email]



In [33]:
handoff_tools

[FunctionTool(name='subject_writer', description='Writes engaging email subject lines based on the email content provided', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11515c400>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Converts a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11515e7a0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_test_email', description='', params_json_schema={'properties': {

In [36]:
email_formatter_instructions = "You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(
    name="Email Manager",
    instructions=email_formatter_instructions,
    tools=handoff_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)



In [39]:
# Now recreating sales manager agent to use handoff tools

sales_manager_instructions = "You're sales manager of the company ComplAI, you use tools given to you to help you write cold sales emails to potential clients. \
    You never write email yourself, you always use the tools given to you. \
    You use all 3 sales agents tool to write 3 different versions of cold sales email, and then choose the best one. \
    You can use tools multiple times if not satisfied with the result. \
    You select the single best email using your own judgement of which email will be most effective. \
    After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(          
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoffs=[emailer_agent]
)

In [40]:
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

API Key exists: True
Email sent successfully!
Status code: 202
Response body: b''
Response headers: Server: nginx
Date: Fri, 22 Aug 2025 21:33:10 GMT
Content-Length: 0
Connection: close
X-Message-Id: nCJEY953Q6mDHaFi80OiQg
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin


